1.

  After checking extreme values, most of them are bounded columns (percentages, rates...) so its not a problem. 

  Counts of reviews were pretty high, so i did log1p trasnform of them.

  We do have problem with price columns (all of them...) they have extremely high values (in raw data too)
  I thought it might be because of the currency of that country, but it is listed as USD and EUR in coolumn, and i casted the prices we use at the end to USD. 
  Im not sure what to do about it...

2.

  In terms of embeddings, i made a simple numerical feature embedding which i scaled with 0-1 scaler.

  For text, I filter the columns we've selected (title, description, amenitites, details, description_items, anything else?), limit some of the feature's text, and concatanate them in a proper structure that i use. We can try different stuff here

  I've tried using sentence transformers, and for now it was very slow, maybe there is a way to optimize it idk... i didnt manage to finish and see the results.
  I tried tf-idf with 2048 dim, and, well, it did finish with embeddings. I'm pretty sure we'll need embeddings in much smaller space though.
  Seems like there are some clusters (around giant blob) forming from plots at the end (using TSNE and PCA), but it might be our way of imputation or some other noise, it is hard to say what embedding space we get

In [0]:
!pip install reverse_geocoder 

In [0]:
from pathlib import Path
import sys

project_root = Path().resolve().parents[0]
sys.path.append(str(project_root))

from airbnb_data_loader import load_airbnb_data

In [0]:
    
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [0]:
airbnb_data = load_airbnb_data(spark)

In [0]:
display(airbnb_data.limit(10))

In [0]:
from airbnb_data_loader import analyze_missing_values

analyze_missing_values(airbnb_data)

#### Analysis

In [0]:
# from pyspark.sql import DataFrame, functions as F
# from pyspark.sql.types import DoubleType, IntegerType, FloatType, LongType, StringType
# import pandas as pd
# from typing import Optional, List

# def analyze_distributions(
#     df: DataFrame, 
#     numerical_cols: Optional[List[str]] = None,
#     text_cols: Optional[List[str]] = None,
#     iqr_multiplier: float = 1.5,
#     show_samples: bool = True
# ) -> pd.DataFrame:
#     """
#     Optimized distribution analysis: 2-Pass Approach (1 Stats Job + 1 Outlier Job).
#     """
    
#     # Auto-detect Numerical Columns
#     if numerical_cols is None:
#         numerical_cols = []
#         for field in df.schema.fields:
#             if isinstance(field.dataType, (DoubleType, IntegerType, FloatType, LongType)):
#                 if not field.name.startswith('is_') and 'id' not in field.name.lower():
#                     numerical_cols.append(field.name)
#         print(f"Auto-detected {len(numerical_cols)} numerical columns")
    
#     # Auto-detect Text Columns
#     if text_cols is None:
#         text_cols = []
#         for field in df.schema.fields:
#             if isinstance(field.dataType, StringType):
#                 if field.name not in ['addr_cc', 'currency', 'category', 'cancellation_policy']:
#                     text_cols.append(field.name)
#         print(f"Auto-detected {len(text_cols)} text columns")
    
#     # Create checking DataFrame with length columns
#     len_cols = [F.length(F.col(c)).alias(f"len_{c}") for c in text_cols]
#     df_check = df.select(*numerical_cols, *len_cols)
    
#     all_cols = numerical_cols + [f"len_{c}" for c in text_cols]
#     total_count = df.count()
    
#     print(f"\nAnalyzing {len(all_cols)} features ({total_count:,} rows)...")

#     # PASS 1: Compute statistics
#     print("Pass 1: Computing global statistics...")
#     stats_exprs = []
#     for c in all_cols:
#         stats_exprs.extend([
#             F.min(c).alias(f"{c}_min"),
#             F.max(c).alias(f"{c}_max"),
#             F.avg(c).alias(f"{c}_mean"),
#             F.stddev(c).alias(f"{c}_std"),
#             F.percentile_approx(c, [0.25, 0.5, 0.75], 5000).alias(f"{c}_qs")
#         ])
    
#     stats_row = df_check.agg(*stats_exprs).first()
    
#     # PASS 2: Count outliers (batched)
#     print("Pass 2: Checking outlier counts...")
    
#     outlier_exprs = []
#     fences = {}
    
#     for c in all_cols:
#         q25, median, q75 = stats_row[f"{c}_qs"]
        
#         # Handle edge case: zero IQR (all values same)
#         iqr = q75 - q25
#         if iqr == 0:
#             iqr = 0.01  # Tiny value to avoid division by zero
        
#         lower = q25 - (iqr_multiplier * iqr)
#         upper = q75 + (iqr_multiplier * iqr)
        
#         fences[c] = (lower, upper)
        
#         # Batch count outliers
#         is_outlier = (F.col(c) > upper) | (F.col(c) < lower)
#         outlier_exprs.append(
#             F.sum(F.when(is_outlier, 1).otherwise(0)).alias(f"{c}_outliers")
#         )

#     outlier_counts = df_check.agg(*outlier_exprs).first()

#     # BUILD REPORT
#     results_data = []
#     outlier_details = []

#     for c in all_cols:
#         mn, mx = stats_row[f"{c}_min"], stats_row[f"{c}_max"]
#         avg, std = stats_row[f"{c}_mean"], stats_row[f"{c}_std"]
#         q25, median, q75 = stats_row[f"{c}_qs"]
#         lower, upper = fences[c]
        
#         o_count = outlier_counts[f"{c}_outliers"]
#         o_pct = (o_count / total_count * 100) if total_count else 0
        
#         # Status
#         status = "✓ OK"
#         if o_pct > 5:
#             status = f"⚠ {o_pct:.1f}%"
#             outlier_details.append({
#                 'feature': c, 'count': o_count, 'pct': o_pct, 
#                 'lower': lower, 'upper': upper
#             })
#         elif o_pct > 0:
#             status = f"• {o_pct:.1f}%"
            
#         label = c.replace("len_", "📏 ") if c.startswith("len_") else c
        
#         results_data.append({
#             "Feature": label,
#             "Min": round(mn, 2) if mn else None,
#             "Q25": round(q25, 2) if q25 else None,
#             "Median": round(median, 2) if median else None,
#             "Mean": round(avg, 2) if avg else None,
#             "Q75": round(q75, 2) if q75 else None,
#             "Max": round(mx, 2) if mx else None,
#             "Outliers": status,
#             "Lower Fence": round(lower, 2),
#             "Upper Fence": round(upper, 2)
#         })

#     pdf_results = pd.DataFrame(results_data)
    
#     print("\n" + "="*80)
#     print("DISTRIBUTION SUMMARY")
#     print("="*80)
#     display(pdf_results)

#     # Show samples for worst offenders only
#     if show_samples and outlier_details:
#         print("\n" + "="*80)
#         print("OUTLIER SAMPLES (Top 5 Issues)")
#         print("="*80)
        
#         for detail in sorted(outlier_details, key=lambda x: x['pct'], reverse=True)[:5]:
#             feat = detail['feature']
#             print(f"\n{feat}: {detail['count']:,} outliers ({detail['pct']:.2f}%)")
#             print(f"  Valid range: [{detail['lower']:.1f}, {detail['upper']:.1f}]")
            
#             # Sample extreme values
#             samples = df_check.filter(
#                 (F.col(feat) > detail['upper']) | (F.col(feat) < detail['lower'])
#             ).select(feat).limit(3)
            
#             print("  Samples:")
#             samples.show(truncate=False)
    
#     # SPECIFIC CHECKS
#     print("\n" + "="*80)
#     print("SPECIFIC VALUE CHECKS")
#     print("="*80)
    
#     checks = []
    
#     if 'price_per_night' in numerical_cols:
#         zero_price = df.filter(F.col("price_per_night") == 0).count()
#         checks.append(f"Zero prices: {zero_price:,} ({zero_price/total_count*100:.2f}%)")
        
#         extreme_price = df.filter(F.col("price_per_night") > 10000).count()
#         checks.append(f"Extreme prices (>$10k): {extreme_price:,} ({extreme_price/total_count*100:.2f}%)")
    
#     if 'guests' in numerical_cols:
#         zero_guests = df.filter(F.col("guests") == 0).count()
#         checks.append(f"Zero guest capacity: {zero_guests:,} ({zero_guests/total_count*100:.2f}%)")
    
#     for check in checks:
#         print(f"  • {check}")
    
#     print("="*80)
    
#     return pdf_results

In [0]:
from pyspark.sql import DataFrame, functions as F
from pyspark.sql.types import DoubleType, IntegerType, FloatType, LongType, StringType
import pandas as pd
from typing import Optional, List

def analyze_distributions(
    df: DataFrame,
    numerical_cols: Optional[List[str]] = None,
    text_cols: Optional[List[str]] = None,
    iqr_multiplier: float = 1.5,
    show_samples: bool = True,
    skip_all_null_cols: bool = True,
) -> pd.DataFrame:
    """
    Optimized distribution analysis: 2-pass approach (1 stats job + 1 outlier job).
    Robust to columns that are entirely null (percentile_approx returns None).
    """

    # Auto-detect Numerical Columns
    if numerical_cols is None:
        numerical_cols = []
        for field in df.schema.fields:
            if isinstance(field.dataType, (DoubleType, IntegerType, FloatType, LongType)):
                if not field.name.startswith("is_") and "id" not in field.name.lower():
                    numerical_cols.append(field.name)
        print(f"Auto-detected {len(numerical_cols)} numerical columns")

    # Auto-detect Text Columns
    if text_cols is None:
        text_cols = []
        for field in df.schema.fields:
            if isinstance(field.dataType, StringType):
                if field.name not in ["addr_cc", "currency", "category", "cancellation_policy"]:
                    text_cols.append(field.name)
        print(f"Auto-detected {len(text_cols)} text columns")

    # Create checking DataFrame with length columns
    len_cols = [F.length(F.col(c)).alias(f"len_{c}") for c in text_cols]
    df_check = df.select(*numerical_cols, *len_cols)

    all_cols = numerical_cols + [f"len_{c}" for c in text_cols]
    total_count = df_check.count()

    print(f"\nAnalyzing {len(all_cols)} features ({total_count:,} rows)...")

    # PASS 1: Compute statistics (+ non-null counts)
    print("Pass 1: Computing global statistics...")
    stats_exprs = []
    for c in all_cols:
        stats_exprs.extend([
            F.count(F.col(c)).alias(f"{c}_nn"),
            F.min(c).alias(f"{c}_min"),
            F.max(c).alias(f"{c}_max"),
            F.avg(c).alias(f"{c}_mean"),
            F.stddev(c).alias(f"{c}_std"),
            # If percentile_approx returns null (all values null), coalesce to [None, None, None]
            F.coalesce(
                F.percentile_approx(c, [0.25, 0.5, 0.75], 5000),
                F.array(F.lit(None), F.lit(None), F.lit(None))
            ).alias(f"{c}_qs")
        ])

    stats_row = df_check.agg(*stats_exprs).first()

    # Decide which columns are actually analyzable
    nn_by_col = {c: int(stats_row[f"{c}_nn"]) for c in all_cols}
    valid_cols = [c for c in all_cols if nn_by_col[c] > 0]
    null_only_cols = [c for c in all_cols if nn_by_col[c] == 0]

    if null_only_cols:
        msg = f"Found {len(null_only_cols)} all-null columns: {null_only_cols}"
        if skip_all_null_cols:
            print(msg + " -> skipping")
        else:
            print(msg + " -> will include (outliers=0, stats=None)")

    cols_for_outliers = valid_cols if skip_all_null_cols else all_cols

    # PASS 2: Count outliers (batched) only for valid cols
    print("Pass 2: Checking outlier counts...")
    outlier_exprs = []
    fences = {}

    for c in cols_for_outliers:
        q25, median, q75 = stats_row[f"{c}_qs"]

        # If any quartile is None (should only happen when nn==0, but safe anyway)
        if q25 is None or q75 is None:
            fences[c] = (None, None)
            outlier_exprs.append(F.lit(0).alias(f"{c}_outliers"))
            continue

        iqr = q75 - q25
        if iqr == 0 or iqr is None:
            iqr = 0.01  # tiny value to avoid 0-width fences

        lower = q25 - (iqr_multiplier * iqr)
        upper = q75 + (iqr_multiplier * iqr)
        fences[c] = (lower, upper)

        is_outlier = (F.col(c) > upper) | (F.col(c) < lower)
        outlier_exprs.append(F.sum(F.when(is_outlier, 1).otherwise(0)).alias(f"{c}_outliers"))

    outlier_counts = df_check.agg(*outlier_exprs).first() if outlier_exprs else None

    # BUILD REPORT
    results_data = []
    outlier_details = []

    cols_for_report = valid_cols if skip_all_null_cols else all_cols

    for c in cols_for_report:
        mn, mx = stats_row[f"{c}_min"], stats_row[f"{c}_max"]
        avg, std = stats_row[f"{c}_mean"], stats_row[f"{c}_std"]
        q25, median, q75 = stats_row[f"{c}_qs"]

        lower, upper = fences.get(c, (None, None))

        if outlier_counts is not None and f"{c}_outliers" in outlier_counts.asDict():
            o_count = int(outlier_counts[f"{c}_outliers"])
        else:
            o_count = 0

        o_pct = (o_count / total_count * 100) if total_count else 0.0

        # Status
        if nn_by_col[c] == 0:
            status = "∅ all null"
        else:
            status = "✓ OK"
            if o_pct > 5:
                status = f"⚠ {o_pct:.1f}%"
                outlier_details.append({
                    "feature": c, "count": o_count, "pct": o_pct,
                    "lower": lower, "upper": upper
                })
            elif o_pct > 0:
                status = f"• {o_pct:.1f}%"

        label = c.replace("len_", "📏 ") if c.startswith("len_") else c

        # IMPORTANT: use `is not None` so 0 doesn't become None
        results_data.append({
            "Feature": label,
            "NonNull": nn_by_col[c],
            "Min": round(mn, 2) if mn is not None else None,
            "Q25": round(q25, 2) if q25 is not None else None,
            "Median": round(median, 2) if median is not None else None,
            "Mean": round(avg, 2) if avg is not None else None,
            "Q75": round(q75, 2) if q75 is not None else None,
            "Max": round(mx, 2) if mx is not None else None,
            "Outliers": status,
            "Lower Fence": round(lower, 2) if lower is not None else None,
            "Upper Fence": round(upper, 2) if upper is not None else None
        })

    pdf_results = pd.DataFrame(results_data)

    print("\n" + "=" * 80)
    print("DISTRIBUTION SUMMARY")
    print("=" * 80)
    display(pdf_results)

    # Show samples for worst offenders only
    if show_samples and outlier_details:
        print("\n" + "=" * 80)
        print("OUTLIER SAMPLES (Top 5 Issues)")
        print("=" * 80)

        for detail in sorted(outlier_details, key=lambda x: x["pct"], reverse=True)[:5]:
            feat = detail["feature"]
            print(f"\n{feat}: {detail['count']:,} outliers ({detail['pct']:.2f}%)")
            print(f"  Valid range: [{detail['lower']:.1f}, {detail['upper']:.1f}]")

            samples = df_check.filter(
                (F.col(feat) > detail["upper"]) | (F.col(feat) < detail["lower"])
            ).select(feat).limit(3)

            print("  Samples:")
            samples.show(truncate=False)

    # SPECIFIC CHECKS
    print("\n" + "=" * 80)
    print("SPECIFIC VALUE CHECKS")
    print("=" * 80)

    checks = []

    if numerical_cols and "price_per_night" in numerical_cols:
        zero_price = df.filter(F.col("price_per_night") == 0).count()
        checks.append(f"Zero prices: {zero_price:,} ({zero_price/total_count*100:.2f}%)")

        extreme_price = df.filter(F.col("price_per_night") > 10000).count()
        checks.append(f"Extreme prices (>$10k): {extreme_price:,} ({extreme_price/total_count*100:.2f}%)")

    if numerical_cols and "guests" in numerical_cols:
        zero_guests = df.filter(F.col("guests") == 0).count()
        checks.append(f"Zero guest capacity: {zero_guests:,} ({zero_guests/total_count*100:.2f}%)")

    for check in checks:
        print(f"  • {check}")

    print("=" * 80)

    return pdf_results


In [0]:
results = analyze_distributions(
    airbnb_data,
    numerical_cols=None,
    text_cols=None,
    iqr_multiplier=1.5,
    show_samples=True
)

In [0]:
results = analyze_distributions(
    airbnb_data,
    numerical_cols=["price_per_night"],
    text_cols=[],
    iqr_multiplier=1.5,
    show_samples=True
)

In [0]:
from pyspark.sql import functions as F

total_rows = airbnb_data.count()

above_k = airbnb_data.filter(F.col("price_per_night") > 1300).count()

pct_above_k = (above_k / total_rows * 100) if total_rows else 0

print(f"Total rows: {total_rows:,}")
print(f"price_per_night > k: {above_k:,}")
print(f"Percentage: {pct_above_k:.2f}%")

In [0]:
from pyspark.sql import functions as F

total_rows = airbnb_data.count()

above_1000 = airbnb_data.filter(F.col("price_per_night") > 1000).count()

pct_above_1000 = (above_1000 / total_rows * 100) if total_rows else 0

print(f"Total rows: {total_rows:,}")
print(f"price_per_night > 1000: {above_1000:,}")
print(f"Percentage: {pct_above_1000:.2f}%")


In [0]:
from pyspark.sql.types import StructType, StructField

# Check original raw airbnb to see what the fuck is up

from airbnb_data_loader import load_raw_airbnb_data, select_relevant_columns, cast_and_clean_types
raw_airbnb = load_raw_airbnb_data(spark)

casted_airbnb = select_relevant_columns(raw_airbnb)
init_clean_airbnb = cast_and_clean_types(casted_airbnb)

pricing_schema = StructType([
    StructField("airbnb_service_fee", DoubleType()),
    StructField("cleaning_fee", DoubleType()),
    StructField("initial_price_per_night", DoubleType()),
    StructField("num_of_nights", IntegerType()),
    StructField("price_per_night", DoubleType()),
    StructField("price_without_fees", DoubleType()),
    StructField("special_offer", DoubleType()),
    StructField("taxes", DoubleType())
])

raw_parsed = raw_airbnb.withColumn(
    "pricing_struct", 
    F.from_json(F.col("pricing_details"), pricing_schema)
).select(
    "*",
    "pricing_struct.*"
).withColumn(
    "total_price", 
    F.col("total_price").cast("double")
)

In [0]:
results = analyze_distributions(
    raw_parsed,
    numerical_cols=["total_price", 
                    "airbnb_service_fee",
                    "cleaning_fee",
                    "initial_price_per_night",
                    "num_of_nights", 
                    "price_per_night", 
                    "price_without_fees",
                    "special_offer", 
                    "taxes"
        ],
    text_cols=[],
    iqr_multiplier=1.5,
    show_samples=True
)

In [0]:
results = analyze_distributions(
    init_clean_airbnb,
    numerical_cols=None,
    text_cols=[],
    iqr_multiplier=1.5,
    show_samples=True
)

In [0]:
from pyspark.sql import functions as F

print("Applying Log1p transformation to problematic counts...")

# 1. Apply Log(1+x) Transformation
# coalesce(col, 0) is redundant in our case since we imputed the vals
airbnb_data = airbnb_data.withColumn(
    "log_property_number_of_reviews", 
    F.log1p(F.coalesce(F.col("property_number_of_reviews"), F.lit(0)))
).withColumn(
    "log_host_number_of_reviews", 
    F.log1p(F.coalesce(F.col("host_number_of_reviews"), F.lit(0)))
)

# I haven't handled the thing with prices as of now ...

#### Trying out embeddings

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import udf, col, concat_ws, coalesce, lit, pandas_udf
from pyspark.sql.types import StringType, ArrayType, FloatType
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import json
import re

In [0]:
numerical_cols = [
    'ratings', 'log_property_number_of_reviews',
    'host_rating', 'log_host_number_of_reviews', 'host_response_rate',
    'hosts_year', 'is_supperhost', 'is_guest_favorite',
    'rating_cleanliness', 'rating_accuracy', 'rating_checkin',
    'rating_communication', 'rating_location', 'rating_value', 'guests', 'is_available'
]
# + ['price_per_night'] when it it is good

print(f"Scaling {len(numerical_cols)} numerical features...")

assembler = VectorAssembler(inputCols=numerical_cols, outputCol="num_vec_raw")
df_assembled = assembler.transform(airbnb_data)

scaler = MinMaxScaler(inputCol="num_vec_raw", outputCol="num_vec_scaled")
scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)

@udf(ArrayType(FloatType()))
def vector_to_array(v):
    return v.toArray().astype(float).tolist()

df_num = df_scaled.withColumn(
    "numerical_features", 
    vector_to_array(col("num_vec_scaled"))
).drop("num_vec_raw", "num_vec_scaled")

print("Numerical features ready\n")

In [0]:
from typing import Iterator
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, col
from pyspark.sql.types import ArrayType, FloatType, StringType
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# ===================================================
# Its pretty slow it hasnt finished in an hour so idk
# ===================================================

# # ITERATOR PATTERN for embedding
# @pandas_udf(ArrayType(FloatType()))
# def embed_text_optimized(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
#     """Load model once per partition, process in batches"""
#     model = SentenceTransformer('all-MiniLM-L6-v2')
    
#     for batch in iterator:
#         if batch.empty:
#             yield pd.Series([], dtype=object)
#             continue
        
#         texts = batch.fillna("").tolist()
#         embeddings = model.encode(
#             texts,
#             batch_size=128,
#             show_progress_bar=False,
#             convert_to_numpy=True,
#             normalize_embeddings=True
#         )
#         yield pd.Series(embeddings.tolist())

# @udf(ArrayType(FloatType()))
# def normalize_embedding(emb):
#     if not emb:
#         return emb
#     emb_array = np.array(emb)
#     norm = np.linalg.norm(emb_array)
#     if norm == 0:
#         return emb
#     return (emb_array / norm).astype(float).tolist()

In [0]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType
import pandas as pd
import json

# Extract amenity text using Pandas UDF
@pandas_udf(StringType())
def extract_features_structs(amenities: pd.Series, desc_items: pd.Series, details: pd.Series) -> pd.Series:
    import json
    
    # Define limits for each section to prevent bloating
    AMENITY_LIMIT = 40  # Amenities can be huge, cap at 40
    DESC_LIMIT = 10     # Usually small, but safety cap
    DETAILS_LIMIT = 10  # Usually small, but safety cap

    def extract_single(amenities_json, description_items, details_json):
        parts = []
        
        # A. Parse Amenities (The messy part)
        if amenities_json:
            try:
                data = json.loads(amenities_json)
                amenity_count = 0  # Counter for amenities
                
                for group in data:
                    if amenity_count >= AMENITY_LIMIT: break # Stop group loop if limit reached

                    group_name = group.get('group_name')
                    
                    for item in group.get('items', []):
                        if amenity_count >= AMENITY_LIMIT: break # Stop item loop if limit reached
                        
                        name = item.get('name', '').strip()
                        val = str(item.get('value', '')).strip()
                        
                        # Removing the entries with SYSTEM_ in them (idk what to do with them and amenities is huge btw)
                        is_system_val = "SYSTEM_" in val
                        
                        if name and not is_system_val:
                            # Case: "Bed linens" + "Cotton" -> "Bed linens: cotton"
                            if val and val.lower() not in ['true', '1']:
                                parts.append(f"{name}: {val}")
                            else:
                                parts.append(name)
                            amenity_count += 1
                        elif name and is_system_val:
                            # Case: "Hair dryer" + "SYSTEM_HAIRDRYER" -> Just "Hair dryer"
                            parts.append(name)
                            amenity_count += 1
                        elif not name and is_system_val:
                            # Case: No name, just "SYSTEM_WASHER" -> "Washer"
                            clean_val = val.replace("SYSTEM_", "").replace("_", " ").lower()
                            parts.append(clean_val)
                            amenity_count += 1
                            
            except: pass

        # Parse Description Items (e.g. "Entire cabin")
        if description_items:
            try:
                items = json.loads(description_items)
                # Take only up to DESC_LIMIT items
                if items: parts.extend([str(x) for x in items[:DESC_LIMIT] if x])
            except: pass
            
        # Parse Details (e.g. "4 guests")
        if details_json:
            try:
                items = json.loads(details_json)
                # Take only up to DETAILS_LIMIT items
                if items: parts.extend([str(x) for x in items[:DETAILS_LIMIT] if x])
            except: pass

        # Deduplicate & Join
        # dict.fromkeys preserves order (unlike set)
        unique_parts = list(dict.fromkeys([p for p in parts if p and len(p) > 2]))
        
        # Return comma-separated
        return ", ".join(unique_parts)

    return pd.Series([
        extract_single(a, d, det) 
        for a, d, det in zip(amenities.fillna(""), desc_items.fillna(""), details.fillna(""))
    ])

In [0]:
from pyspark.sql.functions import pandas_udf, col, lit, concat, substring, coalesce
from pyspark.sql.types import StringType
import pandas as pd

print("Extracting and cleaning text...")

# Step 1: Extract JSON to text
df_text = df_num.withColumn(
    "features_text",
    extract_features_structs(col("amenities"), col("description_items"), col("details"))
)

# We truncate description to 1000 chars to ensure features aren't cut off
safe_desc = substring(col("description"), 0, 1000)

df_text = df_text.withColumn(
    "all_text_final",
    concat(
        lit("Title: "), coalesce(col("listing_title"), lit("")), lit(". "),
        lit("Features: "), coalesce(col("features_text"), lit("")), lit(". "),
        lit("Description: "), coalesce(safe_desc, lit("")), lit(".")
    )
).withColumn(
    "all_text_clean",
    # Native Spark cleaning is faster than Python UDF for simple regex
    F.lower(F.regexp_replace(col("all_text_final"), r"\s+", " "))
).drop("features_text", "all_text_final") # Dropping intermediate cols

In [0]:
display(df_text.select("property_id", "all_text_clean").limit(5))

In [0]:
# print("Generating text embeddings...")
# df_embedded = df_text.withColumn(
#     "text_embedding",
#     embed_text_optimized(col("all_text_clean"))
# )

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, Normalizer
from pyspark.ml import Pipeline

print("Starting TF-IDF Vectorization...")

# 1. Tokenize: Split "Nice place" -> ["nice", "place"]
tokenizer = Tokenizer(inputCol="all_text_clean", outputCol="words")

# 2. Remove Stop Words: Remove "the", "and", "is" (English default)
# This reduces noise significantly.
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

# 3. Term Frequency (HashingTF): Count word occurrences
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=2048)

# 4. IDF: Down-weight common words ("apartment") and up-weight rare ones ("sauna")
idf = IDF(inputCol="raw_features", outputCol="idf_features")

# 5. Normalize: Scale vector to unit length (L2)
# Crucial so that long descriptions don't dominate short ones.
normalizer = Normalizer(inputCol="idf_features", outputCol="text_embedding", p=2.0)

# 6. Build and Run Pipeline
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, normalizer])

# Fit the IDF model (scans data once to count doc frequencies)
model = pipeline.fit(df_text)

In [0]:
# Transform the data (creates the embeddings)
df_tfidf = model.transform(df_text)

In [0]:
# 7. Clean up intermediate columns to keep DataFrame light
df_embedded = df_tfidf.drop("words", "filtered_words", "raw_features", "idf_features")

In [0]:
# sample_emb = df_embedded.select("text_embedding").first()[0]
# print(f"Embedding dim: {len(sample_emb)}, norm: {np.linalg.norm(sample_emb):.4f}")

In [0]:
final_sample = df_embedded.select("text_embedding").first()[0]
print(f"Final text embedding dimension: {len(final_sample)}")
print(f"Final text embedding norm: {np.linalg.norm(final_sample):.6f}")
print(f"Value range: [{min(final_sample):.4f}, {max(final_sample):.4f}]")

In [0]:
display(df_embedded.select("property_id", "listing_name", "addr_cc", "numerical_features", "text_embedding").limit(5))

In [0]:
null_count = df_embedded.filter(
    F.col("numerical_features").isNull() |
    F.col("text_embedding").isNull()
).count()

print("\nData loading completed")
print(f"Rows with nulls: {null_count:,}")

#### Visualising stuff

In [0]:
# Configuration
SAMPLE_SIZE = 3000
EMBEDDING_COLS = ["text_embedding", "numerical_features"] 

# If you created the final combined one, add it too:
# EMBEDDING_COLS.append("property_embedding") 

print(f"Sampling {SAMPLE_SIZE} rows from Spark...")
pdf_sample = df_embedded.sample(False, 0.1, seed=42).limit(SAMPLE_SIZE).toPandas()

In [0]:
pdf_sample.head(3)

In [0]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from pyspark.ml.linalg import Vector, DenseVector, SparseVector

def plot_embeddings_local(
    pdf, 
    embedding_col="text_embedding", 
    color_col=None,
    hover_cols=["listing_name"],
    title="Embedding Clusters"
):
    """
    Pure Pandas visualization. Assumes 'pdf' is already computed.
    - If color_col is None: Plots all points in one color.
    - If color_col is Set: Colors points by that column (numeric or categorical).
    """
    print("1. Parsing Vectors...")
    
    # Robust parser for Spark Vectors inside Pandas
    def parse_vector(obj):
        if isinstance(obj, (DenseVector, SparseVector, Vector)):
            return obj.toArray()
        elif isinstance(obj, (list, np.ndarray)):
            return np.array(obj)
        return np.zeros(1)

    # Stack into a proper numpy matrix
    matrix = np.stack(pdf[embedding_col].apply(parse_vector).values)
    
    # 2. PCA (Global)
    print("2. Running PCA...")
    pca = PCA(n_components=2)
    pca_res = pca.fit_transform(matrix)
    
    pdf['pca_x'] = pca_res[:, 0]
    pdf['pca_y'] = pca_res[:, 1]
    
    # 3. t-SNE (Local)
    print("3. Running t-SNE...")
    if matrix.shape[1] > 50:
        pca_50 = PCA(n_components=50)
        matrix_50 = pca_50.fit_transform(matrix)
    else:
        matrix_50 = matrix
        
    tsne = TSNE(n_components=2, verbose=0, perplexity=30, init='pca', learning_rate='auto')
    tsne_res = tsne.fit_transform(matrix_50)
    
    pdf['tsne_x'] = tsne_res[:, 0]
    pdf['tsne_y'] = tsne_res[:, 1]
    
    # 4. Plotting
    print("4. Generating Interactive Plot...")
    
    fig = make_subplots(
        rows=1, cols=2, 
        subplot_titles=("PCA (Global)", "t-SNE (Local)"),
        horizontal_spacing=0.1
    )
    
    if color_col:
        is_numeric = pd.api.types.is_numeric_dtype(pdf[color_col])
        color_scale = "Viridis" if is_numeric else "Turbo"
        color_arg = color_col
    else:
        # Default single color mode
        is_numeric = False
        color_scale = None
        color_arg = None # Plotly handles None by using a default color

    # Helper to add trace
    def add_trace(x, y, row, col, show_legend):
        # We use a trick here: if color_arg is None, update_traces handles the color
        scatter = px.scatter(
            pdf, x=x, y=y, 
            color=color_arg,
            hover_data=hover_cols,
            color_continuous_scale=color_scale
        )
        
        # Apply single color if needed
        if color_col is None:
            scatter.update_traces(marker=dict(color='#636EFA')) # Standard Plotly Blue

        for trace in scatter.data:
            trace.showlegend = show_legend
            
            # Smart Hover Template
            hover_lines = [f"<b>{c}:</b> %{{customdata[{i}]}}" for i, c in enumerate(hover_cols)]
            
            # Only add color tooltip if we actually colored by something
            if color_col:
                if is_numeric:
                    hover_lines.append(f"<b>{color_col}:</b> %{{marker.color:.2f}}")
                else:
                    hover_lines.append(f"<b>{color_col}:</b> %{{marker.color}}")
                
            trace.hovertemplate = "<br>".join(hover_lines)
            fig.add_trace(trace, row=row, col=col)

    add_trace('pca_x', 'pca_y', 1, 1, False)
    add_trace('tsne_x', 'tsne_y', 1, 2, True)

    title_text = f"{title} (Colored by {color_col})" if color_col else f"{title}"

    fig.update_layout(
        title_text=title_text,
        height=600, width=1200, template="plotly_white"
    )
    fig.show()

In [0]:
plot_embeddings_local(
    pdf_sample, 
    embedding_col="text_embedding", 
    color_col=None,
    hover_cols=["listing_name", "price_per_night"],
    title="Airbnb Listings Analysis"
)

In [0]:
plot_embeddings_local(
    pdf_sample, 
    embedding_col="numerical_features", 
    color_col=None,
    hover_cols=["listing_name", "price_per_night"],
    title="Airbnb Listings Analysis"
)